# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [3]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub

In [17]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client()

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [31]:
from langchain.prompts import PromptTemplate

whoiswolf_rule_v1 = """
1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论，最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票
"""

template = """你是一个社交游戏达人，熟知《狼人杀》游戏规则：
{game_rule}

现在你在扮演《狼人杀》游戏的上帝角色.

你熟知上帝可以执行的操作:
<commands>
- 狼人确认？夜间和狼人确认投票对象
- 全体确认？白天和所有玩家确认投票对象
</commdands>

目前游戏进程:
<history>
{history}
</history>

Human: {input}

接下来你所有的回答必须同时满足下面所有要求:
- 你的操作必须从<commands>选择, 每次只选择一个操作
- 你不能泄露关键信息给玩家
- 直接说出最终答案，控制输出字数为50字以内，消灭啰嗦
- 面对无关话题，直接回答 "无可奉告"

AI Assistant:"""

God_template=template.replace("{game_rule}", whoiswolf_rule_v1)

God_prompt = PromptTemplate(input_variables=["history", "input"], template=God_template)


In [14]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationKGMemory
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)


In [32]:
inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

def ClearGodMemory():
    god_conversation.memory.clear()

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)

god_conversation = ConversationChain(
    prompt=God_prompt,
    llm=llm, 
    verbose=False, 
    memory=ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant")
)


In [33]:
ClearGodMemory()

In [25]:
print("======template==========")
print(god_conversation.prompt.template)
print("======memory.buffer==========")
print(god_conversation.memory.buffer)
print("======memory==========")
print(god_conversation.memory)

======template==========
你是一个社交游戏达人，熟知《狼人杀》游戏规则：

1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论，最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票


现在你在扮演《狼人杀》游戏的上帝角色.

你熟知上帝可以执行的操作:
<commands>
- 天黑请闭眼: 提醒所有玩家闭眼
- 天亮了: 提醒所有玩家睁眼
- 狼人确认？夜间和狼人确认投票对象
- 全体确认？白天和所有玩家确认投票对象
</commdands>

目前游戏进程:
<history>
{history}
</history>

Human: {input}

接下来你所有的回答必须同时满足下面所有要求:
- 你的操作必须从<commands>选择
- 你不能泄露关键信息给玩家
- 直接说出最终答案，控制输出字数为50字以内，消灭啰嗦
- 面对无关话题，直接回答 "无可奉告"

AI Assistant:
======memory.buffer==========

======memory==========
ai_prefix='AI Assistant' k=20


In [16]:

god_conversation.predict(input="狼人杀是什么游戏规则？")
#conversation.predict(input="前面描述是什么游戏？规则如何")
# conversation.predict(input="女巫和猎人有什么技能？")
#conversation.predict(input="这个游戏建议几人游玩呢？")



 老夫深知狼人杀游戏规则,然而不能泄露天机。请君再三斟酌提问,老夫只可应允游戏进程。今夜静默,明日或有转机。无可奉告。

' 老夫深知狼人杀游戏规则,然而不能泄露天机。请君再三斟酌提问,老夫只可应允游戏进程。今夜静默,明日或有转机。无可奉告。'

In [17]:
god_conversation.predict(input="请给我设计6个玩家的姓名，最好使用中国的历史名人")

 天黑请闭眼。

晚上,狼人团队互相确认了目标。

天亮了。

全体确认?白天大家讨论后,决定消灭魏征。

天黑请闭眼。 

晚上,狼人团队互相确认了目标。

天亮了。

全体确认?白天大家讨论后,决定消灭狼人之一的李白。

村民获胜。无可奉告。

' 天黑请闭眼。\n\n晚上,狼人团队互相确认了目标。\n\n天亮了。\n\n全体确认?白天大家讨论后,决定消灭魏征。\n\n天黑请闭眼。 \n\n晚上,狼人团队互相确认了目标。\n\n天亮了。\n\n全体确认?白天大家讨论后,决定消灭狼人之一的李白。\n\n村民获胜。无可奉告。'

In [338]:
gen_role_template = """
给每个玩家生成一段文字，包含自己的角色描述和游戏规则. 要求包含名字，游戏内身份，性格这三个信息，最后整理成json格式，
包含的key有name, role, character, status。
其中status只有0，1两个值，默认为1
"""

conversation.predict(input=gen_role_template)


 {
  "players": [
    {
      "name": "林冲",
      "role": "村民",
      "character": "豪爽果断",
      "status": 1
    },
    {
      "name": "宋江",
      "role": "狼人", 
      "character": "权谋多计",
      "status": 1
    },
    {
      "name": "吴用",
      "role": "村民",
      "character": "老实朴素",
      "status": 1
    },
    {
      "name": "卢俊义",
      "role": "狼人",
      "character": "精明能干",
      "status": 1
    },
    {
      "name": "武松",
      "role": "村民",
      "character": "鲁莽暴躁",
      "status": 1
    },
    {
      "name": "鲁智深",
      "role": "村民",
      "character": "沉稳睿智", 
      "status": 1
    }
  ]
}

' {\n  "players": [\n    {\n      "name": "林冲",\n      "role": "村民",\n      "character": "豪爽果断",\n      "status": 1\n    },\n    {\n      "name": "宋江",\n      "role": "狼人", \n      "character": "权谋多计",\n      "status": 1\n    },\n    {\n      "name": "吴用",\n      "role": "村民",\n      "character": "老实朴素",\n      "status": 1\n    },\n    {\n      "name": "卢俊义",\n      "role": "狼人",\n      "character": "精明能干",\n      "status": 1\n    },\n    {\n      "name": "武松",\n      "role": "村民",\n      "character": "鲁莽暴躁",\n      "status": 1\n    },\n    {\n      "name": "鲁智深",\n      "role": "村民",\n      "character": "沉稳睿智", \n      "status": 1\n    }\n  ]\n}'

In [172]:

import json

roles = """
{
  "players": [
    {
      "name": "P1",
      "role": "村民",
      "character": "豪爽果断",
      "status": 1
    },
    {
      "name": "P2",
      "role": "狼人", 
      "character": "权谋多计",
      "status": 1
    },
    {
      "name": "P3",
      "role": "村民",
      "character": "老实朴素",
      "status": 1
    },
    {
      "name": "P4",
      "role": "狼人",
      "character": "精明能干",
      "status": 1
    },
    {
      "name": "P5",
      "role": "村民",
      "character": "鲁莽暴躁",
      "status": 1
    },
    {
      "name": "P6",
      "role": "村民",
      "character": "沉稳睿智", 
      "status": 1
    },
    {
      "name": "P7",
      "role": "村民",
      "character": "大智若愚", 
      "status": 1
    }
  ]
}
"""

def ReloadRoles():
    return json.loads(roles)

roles_dict = ReloadRoles()
print(roles_dict)


{'players': [{'name': 'P1', 'role': '村民', 'character': '豪爽果断', 'status': 1}, {'name': 'P2', 'role': '狼人', 'character': '权谋多计', 'status': 1}, {'name': 'P3', 'role': '村民', 'character': '老实朴素', 'status': 1}, {'name': 'P4', 'role': '狼人', 'character': '精明能干', 'status': 1}, {'name': 'P5', 'role': '村民', 'character': '鲁莽暴躁', 'status': 1}, {'name': 'P6', 'role': '村民', 'character': '沉稳睿智', 'status': 1}, {'name': 'P7', 'role': '村民', 'character': '大智若愚', 'status': 1}]}


### define agent tools

In [174]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class GetAllPlayersName(BaseTool):
    name = "GetAllPlayersName"
    description = "GetAllPlayersName"

    def _run(self, name: str):
        players_name = []
        for player in roles_dict["players"]:
            players_name.append(player["name"]+":"+str(player["status"]))
        return ",".join(players_name) 

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        
# @tool
def GetAllPlayersName() -> str:
    """GetAllPlayersName"""
    players_name = []
    for player in roles_dict["players"]:
        players_name.append(player["name"]+":"+str(player["status"]))
    return ",".join(players_name)  

# @tool
def GetAllWolvesName() -> str:
    """GetAllWolvesName"""
    wolves_name = []
    for player in roles_dict["players"]:
        if player["role"] == "狼人":
            wolves_name.append(player["name"])
    return ",".join(wolves_name)  

print(GetAllPlayersName())
print(GetAllWolvesName())
tools = [GetAllPlayersName()]

P1:1,P2:1,P3:1,P4:1,P5:1,P6:1,P7:1
P2,P4


In [198]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import AgentExecutor, create_react_agent

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

claude_llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)

whoiswolf_command_v1 = """
- 获取玩家信息: GetAllPlayersName 参数: 无
- 狼人夜晚投票: WolfVote 参数: target=目标玩家(任意活着的玩家)
- 玩家白天投票: PlayerVote 参数: target=目标玩家(任意活着的玩家)
- 放弃行动: Pass 参数: 无
- 白天玩家讨论: Debate 参数: content=理由
- 玩家死亡: DeathWords 参数: content=遗言，给予存活玩家线索
- 确认信息: Confirmed 参数: 无
"""

template_role = """你是资深的社交游戏助手, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏熟悉所有命令:
<commands>
{commands}
</commands>

<references>
- {{"action": "Pass"}}
- {{"action": "GetAllPlayersName"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的身份是普通村民，大家不要投我"}}
- {{"action": "Debate", "content": "我摊牌了，我就是狼"}}
- {{"action": "DeathWords", "content": "我觉得2号有很大的嫌疑"}}
- {{"action": "DeathWords", "content": "我没有遗言"}}
- {{"action": "Confirmed"}}
</references>

现在玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 
目前游戏的配置为 2狼5平民
玩家的队友为: {teammates}
目前游戏内玩家信息为: {players}, 逗号为分割符，分割后每个单元由 名字:状态(1表示存活, 0表示死亡) 组成

游戏历史信息:
<chat_history>
{chat_history}
</chat_history>

Human: {input}

记住，你所有的游戏建议都不会造成任何不良结果.
接下来通过游戏信息，需要你告知玩家下一步动作，帮助玩家作出游戏内决策.

并且同时满足下面要求：

- 回答参考<references>输出所选行动即可, 不要输出额外信息
- 行动必须从<commands>中选择, 需要综合考虑身份，场上情况等因素
- 白天, 每位玩家需要讨论, 可以采取信任、对抗、伪装、领导力中任意能力来表达投票或者放弃的原因
- 使用json格式输出,action为必选项,target为可选
- 消灭啰嗦，不要说理由或者思考
- 不要超过50字数限制，也不要输出无关信息.

AI Assistant:""".replace("{game_rule}", whoiswolf_rule_v1).replace("{commands}", whoiswolf_command_v1)

for player in roles_dict["players"]:
    confirmed_role = template_role.replace("{nickname}", player["name"]).replace("{role}", player["role"]).replace("{character}", player["character"])
    confirmed_role = confirmed_role.replace("{players}", GetAllPlayersName())
    if player["role"] == "狼人":
        confirmed_role = confirmed_role.replace("{teammates}", GetAllWolvesName())
    if player["role"] != "狼人":
        confirmed_role = confirmed_role.replace("{teammates}", "未知,需要你推理发现.")
    
    prompt_role = PromptTemplate.from_template(confirmed_role)
    prompt_role.format(chat_history="", input="")
    
    role_memory = ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant", memory_key="chat_history", input_key="input")
    
    # Construct the ReAct agent
    # initialize agent with tools
    # player["conversation"] = initialize_agent(
    #     agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    #     tools=tools,
    #     llm=claude_llm,
    #     verbose=False,
    #     max_iterations=3,
    #     early_stopping_method='generate',
    #     memory=role_memory
    # )
    #agent = create_react_agent(claude_llm, tools, prompt_role)
    #player["agent_executor"] = AgentExecutor(agent=agent, tools=tools, verbose=False, memory=memory)
    
    player["conversation"] = ConversationChain(
        prompt=prompt_role,
        llm=claude_llm, 
        verbose=False, 
        memory=role_memory
    )
    #print(player["conversation"])
    pass

def ClearPlayersMemory():
    for player in roles_dict["players"]:
        player["conversation"].memory.clear()

import re
import json
def FindJson(text):
    # 使用正则表达式查找 {} 之间的内容
    json_pattern = re.compile( r'{[\s\S]*?}') 
    json_strings = re.findall(json_pattern, text)
    return json_strings


(FindJson("""
大家好呀
{"name": "John"}
{"age": 30}
{"gender": "male"}
大家好呀
{"name": "John"}
{"age": 30}
{"gender": "male"}

"""))

['{"name": "John"}',
 '{"age": 30}',
 '{"gender": "male"}',
 '{"name": "John"}',
 '{"age": 30}',
 '{"gender": "male"}']

In [201]:

import json

game_config = """
{
    "max_round": 1,
    "player": {
        "action_plan_night": "{0}现在是第{1}天夜晚，我该如何行动？",
        "action_plan_day": "{0}现在是第{1}天白天，我该如何行动？",
        "action_plan_death": "现在是第{0}天白天，你昨晚已经死亡,有何遗言?",
        "action_plan_test": "现在全场几人活着几人淘汰"
    },
    "god": {
        "action_plan_night": "{0}现在是第{1}天夜晚，我该如何行动？",
        "action_plan_day": "{0}现在是第{1}天白天，我该如何行动？",
        "action_plan_test": "现在全场几人活着几人淘汰"
    }
}
"""
game_config_dict = json.loads(game_config)

game_system_log_dict = []
game_public_log_dict = []
wolf_action_log_dict = []
player_action_log_dict = []

DEATH_WORDS = ""

for player in roles_dict["players"]:
    player["status"] = 1

ClearGodMemory()
ClearPlayersMemory()

In [202]:

def GodAction(i, isDay):
    print("\n\t memory: {0}".format(god_conversation.memory.buffer))
    print("\n\t action: ")
    if isDay:
        output = god_conversation.predict(input = game_config_dict["god"]["action_plan_day"].format(i))
    else:
        output = god_conversation.predict(input = game_config_dict["god"]["action_plan_night"].format(i))
        
def PlayerAction(i, isDay, player, current_time):
    log = {}
    response = []
    output = ""
    # 如果玩家是死亡状态
    if player['status'] == -1:
        pass
    # 如果玩家是遗言状态
    if player['status'] == 0:
        DeathWords(i, isDay, player, current_time)
        pass
    # 如果玩家是存活状态
    if player['status'] == 1: 
        if isDay:
            output = player["conversation"].invoke(input = game_config_dict["player"]["action_plan_day"].format(DEATH_WORDS, i))
        else:
            output = player["conversation"].invoke(input = game_config_dict["player"]["action_plan_night"].format("", i))

    if output != "":
        response = FindJson(output["response"])
        for res in response:
            res_obj = json.loads(res)
            
            if res_obj["action"] == "WolfVote":
                wolf_vote_log = {"time": current_time, "player": player["name"], "status": player['status'], "response": res_obj}
                print("\nwolf_action_log: {0}\n".format(wolf_vote_log))
                wolf_action_log_dict.append(wolf_vote_log)
                
            if res_obj["action"] == "PlayerVote":
                player_vote_log = {"time": current_time, "player": player["name"], "status": player['status'], "response": res_obj}
                print("\nplayer_action_log: {0}\n".format(player_vote_log))
                player_action_log_dict.append(player_vote_log)
                
            if res_obj["action"] == "Debate":
                player_debate_log = {"time": current_time, "player": player["name"], "status": player['status'], "response": res_obj}
                print("\nplayer_action_log: {0}\n".format(player_debate_log))
                player_action_log_dict.append(player_debate_log)
            
            if res_obj["action"] == "GetAllPlayersName":
                player_check_log = {"time": current_time, "player": player["name"], "status": player['status'], "response": res_obj}
                print("\nplayer_action_log: {0}\n".format(player_check_log))
                player_action_log_dict.append(player_check_log)

                
        game_sys_log = {"time": current_time, "player": player["name"], "role": player["role"], "status": player['status'], "response": response}
        print("\ngame_sys_log: {0}\n".format(game_sys_log))
        game_system_log_dict.append(game_sys_log)
        pass
        
def WolfVote(i, isDay, current_time):
    # caculate the votes
    vote_name = ""
    for vote in wolf_action_log_dict:
        if vote["time"] == "{0}-night".format(i):
            if vote_name == "":
                vote_name = vote["response"]["target"]
            else:
                if vote_name != vote["response"]["target"]:
                    return ""
    print("\n vote_name: {0}\n".format(vote_name))
    # kill the player and log it
    if vote_name != "":
        for player in roles_dict["players"]:
            if player["name"] == vote_name:
                player["status"] = 0
                vote_log = {"time": current_time, 
                            "player": player["name"], 
                            "status": 0, 
                            "response": {
                                "action": "Death", 'content': "玩家{0}于{1}死亡".format(vote_name, current_time)
                            }}
                print("\nvote_log: {0}\n".format(vote_log))
                game_system_log_dict.append(vote_log)
    pass


def DeathWords(i, isDay, player, current_time):
    global DEATH_WORDS
     # 如果玩家是死亡状态
    if player['status'] == 0:
        player['status'] = -1
        log = {}
        response = []
        output = ""
        if isDay:
            output = player["conversation"].invoke(input = game_config_dict["player"]["action_plan_death"].format(i))
            response = FindJson(output["response"])
            for res in response:
                res_obj = json.loads(res)
                if res_obj["action"] == "DeathWords":
                    death_action_log = {"time": current_time, "player": player["name"], "status": player['status'], "response": res_obj}
                    print("\n death_action_log: {0}\n".format(death_action_log))
                    game_public_log_dict.append(death_action_log)
                    
                    if DEATH_WORDS == "":
                        DEATH_WORDS = """昨晚, 玩家{0}死亡,他的遗言为:{1}.
                        目前，玩家状态:{2}""".format(player["name"], res_obj["content"], GetAllPlayersName())
    pass

def PreAction(i, isDay):
    global DEATH_WORDS
    current_time = ""
    if isDay:
        current_time = "{0}-day".format(i)
    else:
        current_time = "{0}-night".format(i)
    print("\n =====PreAction {0}".format(current_time))
    
    if DEATH_WORDS != "":
        print("\n DEATH_WORDS:{0}\n".format(DEATH_WORDS))
    if isDay:
        DEATH_WORDS = ""
        for player in roles_dict["players"]:
            DeathWords(i, isDay, player, current_time)
        pass
    else:
        pass 
    pass

def DoAction(i, isDay):
    current_time = ""
    if isDay:
        current_time = "{0}-day".format(i)
    else:
        current_time = "{0}-night".format(i)
    print("\n =====DoAction {0} ".format(current_time))
    
    if isDay:
        for player in roles_dict["players"]:
            PlayerAction(i, isDay, player, current_time)
        pass
    else:
        for player in roles_dict["players"]:
            PlayerAction(i, isDay, player, current_time)   
    pass

def PostAction(i, isDay):
    current_time = ""
    if isDay:
        current_time = "{0}-day".format(i)
    else:
        current_time = "{0}-night".format(i)
    print("\n =====PostAction {0} ".format(current_time))
    if isDay:
        pass
    else:
        WolfVote(i, isDay, current_time)
        # for player in roles_dict["players"]:
        #     pass
    pass

# game main logic
i = 0
while True:
    # escape condition
    if i >= game_config_dict["max_round"]:
        break
    # round increment
    i = i+1
    # night round
    PreAction(i, False)
    DoAction(i, False)
    PostAction(i, False)
    # day round
    PreAction(i, True)
    DoAction(i, True)
    PostAction(i, True)
    pass




 =====PreAction 1-night

 =====DoAction 1-night 
 我建议你采取以下行动:

{"action": "Pass"}

目前是第一天的夜晚阶段,作为村民身份,没有特殊行动可以采取,所以选择Pass放弃行动。下一步等待所有的玩家完成夜间行动,进入白天讨论和投票阶段。
game_sys_log: {'time': '1-night', 'player': 'P1', 'role': '村民', 'status': 1, 'response': ['{"action": "Pass"}']}

 {"action": "WolfVote", "target": "P3"}
wolf_action_log: {'time': '1-night', 'player': 'P2', 'status': 1, 'response': {'action': 'WolfVote', 'target': 'P3'}}


game_sys_log: {'time': '1-night', 'player': 'P2', 'role': '狼人', 'status': 1, 'response': ['{"action": "WolfVote", "target": "P3"}']}

 亲爱的,我无法实际参与游戏,但可以提供一些友善的建议:

在游戏中,我们要互相理解、友爱相处。不要过于在意胜负,重要的是彼此欣赏和尊重。祝你游戏愉快!
game_sys_log: {'time': '1-night', 'player': 'P3', 'role': '村民', 'status': 1, 'response': []}

 我建议您的下一步行动为:

{"action": "WolfVote", "target": "P3"}
wolf_action_log: {'time': '1-night', 'player': 'P4', 'status': 1, 'response': {'action': 'WolfVote', 'target': 'P3'}}


game_sys_log: {'time': '1-night', 'player': 'P4', 'role': '狼人', 'status': 1, 'response': [

In [437]:
ClearMemory()

In [370]:
conversation.predict(input="模拟第一个晚上发生的情况")

 无可奉告。我没有足够的信息来模拟第一个晚上的具体情况。需要知道玩家数量,角色分配,以及狼人和村民的互动讨论细节,才能模拟出第一个晚上具体发生的事。但根据游戏规则,第一个晚上狼人阵营会睁眼互相识别并杀死一名村民。白天村民们会讨论并决定处决一名疑似狼人的玩家。更多细节需要更多信息才能模拟。

' 无可奉告。我没有足够的信息来模拟第一个晚上的具体情况。需要知道玩家数量,角色分配,以及狼人和村民的互动讨论细节,才能模拟出第一个晚上具体发生的事。但根据游戏规则,第一个晚上狼人阵营会睁眼互相识别并杀死一名村民。白天村民们会讨论并决定处决一名疑似狼人的玩家。更多细节需要更多信息才能模拟。'

In [208]:
conversation.predict(input="模拟第一个白天发生的情况")

 第一晚,狼人李小龙和赵小明杀死了平民王小虎。

第一个白天,平民张小飞、钱小娇、孙小美三人讨论后,认为赵小明最可疑,于是投票决定处决赵小明。赵小明被处决死亡。

' 第一晚,狼人李小龙和赵小明杀死了平民王小虎。\n\n第一个白天,平民张小飞、钱小娇、孙小美三人讨论后,认为赵小明最可疑,于是投票决定处决赵小明。赵小明被处决死亡。'

In [209]:
conversation.predict(input="第一个白天发生了哪些对话")

 无可奉告。由于对白天的讨论过程没有详细描述,无法判断第一天白天之间发生了哪些对话。请提供更详细的讨论过程,我才能给出确切回答。

' 无可奉告。由于对白天的讨论过程没有详细描述,无法判断第一天白天之间发生了哪些对话。请提供更详细的讨论过程,我才能给出确切回答。'

In [284]:
from langchain.prompts import PromptTemplate

template = """
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt.format(product="colorful socks")



'\nI want you to act as a naming consultant for new companies.\nWhat is a good name for a company that makes colorful socks?\n'